# Besoin2

## implémentation random forest from scratch

In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
import json
import pickle
import matplotlib.pyplot as plt

data_prof = pd.read_csv('./Données/Data_Arbre.csv')
data_clean = pd.read_csv("./Données/data_clean.csv", encoding='utf-8', sep=";", decimal=",")

 # Sélectionner les colonnes pertinentes
selected_columns = ['haut_tot', 'haut_tronc', 'tronc_diam', 'fk_stadedev', 'fk_nomtech']
X = data_prof[selected_columns]
y = data_prof['age_estim']
X.shape

# Sélectionner les colonnes catégorielles à encoder
categorical_columns = ['fk_stadedev', 'fk_nomtech']

# Appliquer l'encodage sur les colonnes catégorielles
X_encoded = pd.get_dummies(X, columns=categorical_columns, drop_first=True)

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)
X_encoded.shape

# Normaliser les données
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

#  Créer et entraîner le modèle de régression linéaire
model = LinearRegression()
model.fit(X_train, y_train)
X_train.shape

# Faire des prédictions sur l'ensemble de test
y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R^2 Score: {r2}")

#  Afficher les coefficients du modèle
coefficients = pd.DataFrame(model.coef_, columns=['Coefficient'], index=X_encoded.columns)
print(coefficients)

Mean Squared Error: 5.237086588143302e+20
R^2 Score: -1.3730799368751908e+18
                       Coefficient
haut_tot                  0.227726
haut_tronc                2.285528
tronc_diam                0.167131
fk_stadedev_Jeune        -8.396638
fk_stadedev_senescent    29.282950
...                            ...
fk_nomtech_ULMJAP         4.392011
fk_nomtech_ULMMIN        12.667372
fk_nomtech_ULMRES        14.508682
fk_nomtech_ULMRESreb      0.237516
fk_nomtech_ULMRESsap      0.000000

[225 rows x 1 columns]


random forest

In [3]:
# Créer et entraîner le modèle de forêts aléatoires
model_rf = RandomForestRegressor(n_estimators=100, random_state=42)
model_rf.fit(X_train_scaled, y_train)

# Faire des prédictions sur l'ensemble de test
y_pred_rf = model_rf.predict(X_test_scaled)

# Évaluer le modèle

mse_rf = mean_squared_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)
rmse_rf = np.sqrt(mse_rf)

print(f"Random Forest - Mean Squared Error: {mse_rf}")
print(f"Random Forest - R^2 Score: {r2_rf}")
print(f"Random Forest - RMSE: {rmse_rf}")

Random Forest - Mean Squared Error: 76.82860361273896
Random Forest - R^2 Score: 0.7985677486455527
Random Forest - RMSE: 8.765192731066383


CART

In [10]:
# Créer et entraîner le modèle CART
model_cart = DecisionTreeRegressor(max_depth=10, min_samples_split=2, random_state=42)
model_cart.fit(X_train_scaled, y_train)

# Faire des prédictions sur l'ensemble de test
y_pred_cart = model_cart.predict(X_test_scaled)

# Évaluer le modèle
mse_cart = mean_squared_error(y_test, y_pred_cart)
r2_cart = r2_score(y_test, y_pred_cart)
rmse_cart = np.sqrt(mse_cart)

print(f"CART - Mean Squared Error: {mse_cart}")
print(f"CART - R^2 Score: {r2_cart}")
print(f"CART - RMSE: {rmse_cart}")

CART - Mean Squared Error: 97.84297209297588
CART - R^2 Score: 0.743471451762393
CART - RMSE: 9.891560650017563


# tests


## test cart

In [16]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

#test CART


# Définition de la fonction de variance pour la régression
def variance(y):
    return np.var(y)


# Définition de la fonction de division des noeuds pour la régression
def split(X, y, feature_index, threshold):
    left_indices = np.where(X[:, feature_index] <= threshold)[0]
    right_indices = np.where(X[:, feature_index] > threshold)[0]
    return X[left_indices], X[right_indices], y[left_indices], y[right_indices]


# Définition de la classe Node pour l'arbre de régression
class Node:
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, value=None):
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value


# Définition de la classe DecisionTree pour l'arbre de régression
class DecisionTree:
    def __init__(self, min_samples_split=2, max_depth=2, criterion='variance'):
        self.root = None
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.criterion = criterion

    def fit(self, X, y):
        self.root = self._grow_tree(X, y)

    def _grow_tree(self, X, y, depth=0):
        n_samples, n_features = X.shape
        if (depth >= self.max_depth or n_samples < self.min_samples_split or len(set(y)) == 1):
            leaf_value = np.mean(y)
            return Node(value=leaf_value)

        best_feature, best_threshold = self._best_split(X, y, n_features)

        left_X, right_X, left_y, right_y = split(X, y, best_feature, best_threshold)

        left_child = self._grow_tree(left_X, left_y, depth + 1)
        right_child = self._grow_tree(right_X, right_y, depth + 1)

        return Node(best_feature, best_threshold, left_child, right_child)

    def _best_split(self, X, y, n_features):
        best_var = np.inf
        split_idx, split_thresh = None, None

        for feature_index in range(n_features):
            thresholds = np.unique(X[:, feature_index])

            for threshold in thresholds:
                var = self._variance_index(X, y, feature_index, threshold)

                if var < best_var:
                    best_var = var
                    split_idx = feature_index
                    split_thresh = threshold

        return split_idx, split_thresh

    def _variance_index(self, X, y, feature_index, threshold):
        left_indices = np.where(X[:, feature_index] <= threshold)[0]
        right_indices = np.where(X[:, feature_index] > threshold)[0]

        if len(left_indices) == 0 or len(right_indices) == 0:
            return np.inf

        left_var = variance(y[left_indices])
        right_var = variance(y[right_indices])

        n_left, n_right = len(left_indices), len(right_indices)
        weighted_var = (n_left * left_var + n_right * right_var) / (n_left + n_right)

        return weighted_var

    def predict(self, X):
        return np.array([self._predict(inputs) for inputs in X])

    def _predict(self, inputs):
        node = self.root
        while node.value is None:
            if inputs[node.feature_index] <= node.threshold:
                node = node.left
            else:
                node = node.right
        return node.value


# Génération de données pour l'exemple de régression
X, y = make_regression(n_samples=1000, n_features=20, noise=0.1, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Création du modèle DecisionTree pour la régression
model_regression = DecisionTree(min_samples_split=2, max_depth=10, criterion='variance')

# Entraînement du modèle sur les données d'entraînement
model_regression.fit(X_train, y_train)

# Prédictions sur les données de test
y_pred_regression = model_regression.predict(X_test)

# Évaluation du modèle
mse_regression = mean_squared_error(y_test, y_pred_regression)
r2_regression = r2_score(y_test, y_pred_regression)
rmse_regression = np.sqrt(mse_regression)

print(f"Regression - Mean Squared Error: {mse_regression}")
print(f"Regression - R^2 Score: {r2_regression}")
print(f"Regression - RMSE: {rmse_regression}")


Regression - Mean Squared Error: 16766.08356748525
Regression - R^2 Score: 0.5668651854282613
Regression - RMSE: 129.48391238870275


## tests random forest

In [18]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error, r2_score

class RandomForestClassifier:
    def __init__(self, n_estimators=100, max_depth=None, min_samples_split=2, max_features=None):
        self.n_estimators = n_estimators  # nombre d'arbres dans la forêt
        self.max_depth = max_depth  # profondeur maximale des arbres
        self.min_samples_split = min_samples_split  # nombre minimum d'échantillons requis pour diviser un nœud
        self.max_features = max_features  # nombre de caractéristiques à considérer pour trouver la meilleure division
        self.trees = []  # liste pour stocker les arbres
        self.selected_features = []  # liste pour stocker les indices des caractéristiques sélectionnées pour chaque arbre

    def fit(self, X, y):
        n_samples, n_features = X.shape  # obtient le nombre d'échantillons et de caractéristiques
        if not self.max_features:
            self.max_features = int(np.sqrt(n_features))  # définit max_features à la racine carrée du nb carcateristique si non spécifié
       
        # construction de chaque arbre
        for _ in range(self.n_estimators):
            # bootstrap sampling
            indices = np.random.choice(n_samples, n_samples, replace=True)  # sélectionne des échantillons aléatoires avec remise
            X_bootstrap = X[indices]  # on crée l'échantillon bootstrap pour les caractéristiques
            y_bootstrap = y[indices]  # on crée l'échantillon bootstrap pour les étiquettes

            # sélection aléatoire des caractéristques
            selected_features = np.random.choice(n_features, self.max_features, replace=False)  # sélectionne aléatoirement des caractéristiques 
            self.selected_features.append(selected_features)  # sauvegarde les indices des caractéristiques sélectionnées

            # création et entraînement de l'arbre de décision avec scikit-learn
            tree = DecisionTreeClassifier(max_depth=self.max_depth, min_samples_split=self.min_samples_split)  # crée un arbre de décision
            tree.fit(X_bootstrap[:, selected_features], y_bootstrap)  # entraîne l'arbre sur l'échantillon bootstrap et les caractéristiques sélectionnées

            self.trees.append(tree)  # ajoute l'arbre à la forêt

    def predict(self, X):
        # agréger les prédictions de chaque arbre
        predictions = np.zeros((X.shape[0], len(self.trees)))  # crée un tableau pour stocker les prédictions de chaque arbre pour chaque échantillon
        for i, tree in enumerate(self.trees):
            selected_features = self.selected_features[i]  # récupère les caractéristiques sélectionnées pour cet arbre
            predictions[:, i] = tree.predict(X[:, selected_features])  # prédictions de l'arbre en utilisant les caractéristiques sélectionnées
        
        # vote majoritaire
        return np.round(np.mean(predictions, axis=1)).astype(int)  # calcule la moyenne des prédictions pour chaque échantillon et arrondit pour obtenir les classes finales

# Exemple d'utilisation
X, y = make_classification(n_samples=1000, n_features=20, random_state=42)  # Génère un ensemble de données de classification avec 1000 échantillons et 20 caractéristiques
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)  # Divise les données en ensembles d'entraînement et de test (80% - 20%)

# Création du modèle de Random Forest
rf_classifier = RandomForestClassifier(n_estimators=100, max_depth=None, min_samples_split=2, max_features=None)  # Crée une instance de RandomForestClassifier avec 100 arbres

# entraînement du modèle sur les données d'entraînement
rf_classifier.fit(X_train, y_train)  # Entraîne le modèle sur les données d'entraînement

# prédictions sur les données de test
y_pred = rf_classifier.predict(X_test)  # Utilise le modèle pour prédire les classes des échantillons de test

# Évaluation supplémentaire avec MSE, R2 Score et RMSE
mse_rf = mean_squared_error(y_test, y_pred)
r2_rf = r2_score(y_test, y_pred)
rmse_rf = np.sqrt(mse_rf)

print(f"Random Forest - Mean Squared Error: {mse_rf}")
print(f"Random Forest - R^2 Score: {r2_rf}")
print(f"Random Forest - RMSE: {rmse_rf}")


Random Forest - Mean Squared Error: 0.1
Random Forest - R^2 Score: 0.5980303487086724
Random Forest - RMSE: 0.31622776601683794


## CART v1 classification

In [16]:
import numpy as np
from collections import Counter

#calcul des critères de division

# Définition de la fonction gini pour la classifiction pour un ensemble de labels y
def gini(y):
    m = len(y)
    return 1.0 - sum((np.sum(y == c) / m) ** 2 for c in np.unique(y))

#ou variance pour la régression
def variance(y):
    return np.var(y)

# Définition de la fonction split de divisions des noeuds
#fonction pour diviser les données basées sur une caractéristique et un seuil
def split(X, y, feature_index, threshold):
    left_indices = np.where(X[:, feature_index] <= threshold)[0]
    right_indices = np.where(X[:, feature_index] > threshold)[0]
    return X[left_indices], X[right_indices], y[left_indices], y[right_indices]

#Construction de l'arbre de décision 

# Définition de la classe Node: noeud de l'abre de décision
class Node:
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, value=None):
        self.feature_index = feature_index  #index de la caractéristique utilisée pour la division
        self.threshold = threshold          #seuil de division
        self.left = left                    #sous-arbre gauche
        self.right = right                  #sous-arbre droit
        self.value = value                  #valeur de la feuille (si c'est une feuille)


# Définition de la classe DecisionTree responsable de la construction et de l'utilisation de l'arbre de décision
class DecisionTree:
    def __init__(self, min_samples_split=2, max_depth=2, criterion='gini'): 
        self.root = None #racine de l'abre
        self.min_samples_split = min_samples_split #nombre minimum d'échantillons pour diviser un nœud
        self.max_depth = max_depth #profondeur maximale de l'arbre
        self.criterion = criterion #critère de division (ici nous utilisons Gini)
    
    def fit(self, X, y):
        self.n_classes = len(set(y)) #nombre de classes uniques 
        self.root = self._grow_tree(X, y)#construction de l'arbre
    
    def _grow_tree(self, X, y, depth=0):
        # vérifie plusieurs conditions pour déterminer s'il faut arrêter la croissance de l'arbre et créer une feuille
        n_samples, n_features = X.shape #initialisation
        #critère d'arret
        if (depth >= self.max_depth or n_samples < self.min_samples_split or len(set(y)) == 1):
            #critères de profondeur maximale, de nombre minimumd'echantillons, appartenance des classes
            leaf_value = self._most_common_label(y) #si l'un des critères est satisfait, une feuille est créé
            return Node(value=leaf_value)
        
        # trouver la meilleure division (best split)
        best_feature, best_threshold = self._best_split(X, y, n_features)
        
        #division des données
        left_X, right_X, left_y, right_y = split(X, y, best_feature, best_threshold)
       
        #construire les sous arbres par récursivité 
        left_child = self._grow_tree(left_X, left_y, depth + 1)
        right_child = self._grow_tree(right_X, right_y, depth + 1)
        return Node(best_feature, best_threshold, left_child, right_child)
    
    #trouve la meilleure caractéristique et le meilleur seuil pour diviser les données
    def _best_split(self, X, y, n_features):
        #initialisation des variables
        best_gini = 1.0 #initialisation à 1 soit la pire valeur possible pour l'impureté de Gini
        split_idx, split_thresh = None, None # stockeront l'index de la caractéristique et  le seuil correspondant à la meilleure division
        
        
        for feature_index in range(n_features):
            thresholds = np.unique(X[:, feature_index]) #tableau des valeurs uniques dans la colonne feature_index de X
            for threshold in thresholds: #boucle parcourt chaque valeur unique (seuil) pour la caractéristique courante
                gini = self._gini_index(X, y, feature_index, threshold)
                #mise à jour de la meilleure division pour les 3 variables
                if gini < best_gini: #Si l'impureté de Gini pour la division courante est meilleure que best_gini, on met à jour best_gini, split_idx et split_thresh
                    best_gini = gini
                    split_idx = feature_index
                    split_thresh = threshold
        return split_idx, split_thresh
    
    def _gini_index(self, X, y, feature_index, threshold): #calcule l'impureté de Gini pour une division donnée par feature_index et threshold
       #on divise les indices 
        left_indices = np.where(X[:, feature_index] <= threshold)[0] #indices des échantillons pour lesquels la valeur de la caractéristique est inférieure ou égale au seuil
        right_indices = np.where(X[:, feature_index] > threshold)[0] # indices des échantillons pour lesquels la valeur de la caractéristique est supérieure au seuil
        #vérification des divisions vides
        if len(left_indices) == 0 or len(right_indices) == 0:#si une des divisions (gauche ou droite) est vide-->
            return 1.0  #retourner 1.0 =la pire impureté de Gini (division inutile)
       
        #on calcule de l'Impureté de Gini pour chaque division
        left_gini = gini(y[left_indices]) # impureté de Gini pour les labels des échantillons dans la division gauche
        right_gini = gini(y[right_indices])# "" droite
        #calcul des poids pour les divisions
        n_left, n_right = len(left_indices), len(right_indices)
        #calcul de l'impureté de Gini pondérée par le nombre d'echantillons dans chaque division
        weighted_gini = (n_left * left_gini + n_right * right_gini) / (n_left + n_right)
        
        return weighted_gini
    
    def _most_common_label(self, y):
        return Counter(y).most_common(1)[0][0] #retourne le label le plus fréquent dans un ensemble de labels y
    
    def predict(self, X):#utilise l'arbre de décision pour faire des prédictions sur un ensmeble de nouvelles données X
        return np.array([self._predict(inputs) for inputs in X]) #applique la fonction _predict sur chaque exemple d'entrée dans X et retourne un tableau des prédictions
    
    def _predict(self, inputs):# traverse l'arbre de décision pour prédire la classe pour un exemple d'entrée donné
        node = self.root #initialisation
        #on traverse l'arbre
        while node.value is None: #Tant que le nœud courant n'est pas une feuille (n'a pas de valeur de classe), la fonction vérifie la valeur de la caractéristique de inputs correspondant à node.feature_index
            if inputs[node.feature_index] <= node.threshold: #si cette valeur est inférieure ou égale à node.threshold elle se déplace vers le sous-arbre gauche (node.left), sinon elle se déplace vers le sous-arbre droit (node.right).
                node = node.left
            else:
                node = node.right
        return node.value #retourne la prédiction

# test avec de données random pour l'exemple
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X, y = make_classification(n_samples=1000, n_features=20, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#on évalue des performances
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")



Accuracy: 0.875


## CART v2 regression commenter!!!

In [21]:
import numpy as np
from collections import Counter
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Définition de la fonction de variance pour la régression
def variance(y):
    return np.var(y)

# Définition de la fonction de division des noeuds pour la régression
def split(X, y, feature_index, threshold):
    left_indices = np.where(X[:, feature_index] <= threshold)[0]
    right_indices = np.where(X[:, feature_index] > threshold)[0]
    return X[left_indices], X[right_indices], y[left_indices], y[right_indices]

# Définition de la classe Node pour l'arbre de régression
class Node:
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, value=None):
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

# Définition de la classe DecisionTree pour l'arbre de régression
class DecisionTree:
    def __init__(self, min_samples_split=2, max_depth=2, criterion='variance'):
        self.root = None
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.criterion = criterion

    def fit(self, X, y):
        self.root = self._grow_tree(X, y)

    def _grow_tree(self, X, y, depth=0):
        n_samples, n_features = X.shape
        if (depth >= self.max_depth or n_samples < self.min_samples_split or len(set(y)) == 1):
            leaf_value = np.mean(y)
            return Node(value=leaf_value)

        best_feature, best_threshold = self._best_split(X, y, n_features)

        left_X, right_X, left_y, right_y = split(X, y, best_feature, best_threshold)

        left_child = self._grow_tree(left_X, left_y, depth + 1)
        right_child = self._grow_tree(right_X, right_y, depth + 1)

        return Node(best_feature, best_threshold, left_child, right_child)

    def _best_split(self, X, y, n_features):
        best_var = np.inf
        split_idx, split_thresh = None, None

        for feature_index in range(n_features):
            thresholds = np.unique(X[:, feature_index])

            for threshold in thresholds:
                var = self._variance_index(X, y, feature_index, threshold)

                if var < best_var:
                    best_var = var
                    split_idx = feature_index
                    split_thresh = threshold

        return split_idx, split_thresh

    def _variance_index(self, X, y, feature_index, threshold):
        left_indices = np.where(X[:, feature_index] <= threshold)[0]
        right_indices = np.where(X[:, feature_index] > threshold)[0]

        if len(left_indices) == 0 or len(right_indices) == 0:
            return np.inf

        left_var = variance(y[left_indices])
        right_var = variance(y[right_indices])

        n_left, n_right = len(left_indices), len(right_indices)
        weighted_var = (n_left * left_var + n_right * right_var) / (n_left + n_right)

        return weighted_var

    def predict(self, X):
        return np.array([self._predict(inputs) for inputs in X])

    def _predict(self, inputs):
        node = self.root
        while node.value is None:
            if inputs[node.feature_index] <= node.threshold:
                node = node.left
            else:
                node = node.right
        return node.value


NameError: name 'r2_score' is not defined

NameError: name 'r2_score' is not defined

## CART BON Emilie

## random forest

In [18]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

class RandomForestClassifier:
    def __init__(self, n_estimators=100, max_depth=None, min_samples_split=2, max_features=None):
        self.n_estimators = n_estimators  # nombre d'arbres dans la forêt
        self.max_depth = max_depth  #profondeur maximale des arbres
        self.min_samples_split = min_samples_split  #nombre minimum d'échantillons requis pour diviser un nœud
        self.max_features = max_features  # nombre de caractéristiques à considérer pour trouver la meilleure division
        self.trees = []  # liste pour stocker les arbres
        self.selected_features = []  # liste pour stocker les indices des caractéristiques sélectionnées pour chaque arbre

    def fit(self, X, y): #méthode fit
        n_samples, n_features = X.shape  #obtient le nombre d'échantillons et de caractéristiques
        if not self.max_features:
            self.max_features = int(np.sqrt(n_features))  # définit max_features à la racine carrée du nb carcateristique si non spécifié
       
        #construction de chaque arbre
        for _ in range(self.n_estimators):
            #bootstrap sampling
            indices = np.random.choice(n_samples, n_samples, replace=True)  #sélectionne des échantillons aléatoires avec remise
            X_bootstrap = X[indices]  #on crée l'échantillon bootstrap pour les caractéristiques
            y_bootstrap = y[indices]  #on crée l'échantillon bootstrap pour les étiquettes

            # sélection aléatoire des caractéristques
            selected_features = np.random.choice(n_features, self.max_features, replace=False)  #sélectionne aléatoirement des caractéristiques 
            self.selected_features.append(selected_features)  #sauvegarde les indices des caractéristiques sélectionnées

            # création et entraînement de l'arbre de décision avec scikit-learn
            tree = DecisionTreeClassifier(max_depth=self.max_depth, min_samples_split=self.min_samples_split)  #crée un arbre de décision
            tree.fit(X_bootstrap[:, selected_features], y_bootstrap)  #entraîne l'arbre sur l'échantillon bootstrap et les caractéristiques sélectionnées

            self.trees.append(tree)  # ajoute l'arbre à la forêt


    def predict(self, X):
        # agréger les prédictions de chaque arbre
        predictions = np.zeros((X.shape[0], len(self.trees)))  #crée un tableau pour stocker les prédictions de chaque arbre pour chaque échantillon
        for i, tree in enumerate(self.trees):
            selected_features = self.selected_features[i]  #récupère les caractéristiques sélectionnées pour cet arbre
            predictions[:, i] = tree.predict(X[:, selected_features])  #prédictions de l'arbre en utilisant les caractéristiques sélectionnées
        
        # vote majoritaire
        return np.round(np.mean(predictions, axis=1)).astype(int)  #calcule la moyenne des prédictions pour chaque échantillon et arrondit pour obtenir les classes finales

# Exemple d'utilisation

# test données random
X, y = make_classification(n_samples=1000, n_features=20, random_state=42)  # Génère un ensemble de données de classification avec 1000 échantillons et 20 caractéristiques
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)  # Divise les données en ensembles d'entraînement et de test (80% - 20%)

# Création du modèle de Random Forest
rf_classifier = RandomForestClassifier(n_estimators=100, max_depth=None, min_samples_split=2, max_features=None)  # Crée une instance de RandomForestClassifier avec 100 arbres

#entraînement du modèle sur les données d'entraînement
rf_classifier.fit(X_train, y_train)  # Entraîne le modèle sur les données d'entraînement

#prédictions sur les données de test
y_pred = rf_classifier.predict(X_test)  # Utilise le modèle pour prédire les classes des échantillons de test

#évaluation des performances
accuracy = accuracy_score(y_test, y_pred)  # Évalue la performance en calculant l'exactitude des prédictions
print(f"Accuracy: {accuracy}")  # Affiche l'exactitude des prédictions



Accuracy: 0.83
